In [1]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.5 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import pandas as pd

In [4]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='county-coordinate-collection')

In [5]:
# Function to add centroid coordinates as properties
def add_centroid(feature):
    centroid = feature.geometry().centroid()
    return feature.set({
        'centroid_lon': centroid.coordinates().get(0),
        'centroid_lat': centroid.coordinates().get(1),
        'county_name': feature.get('ADM2_NAME')
    })

In [11]:
# Load the India administrative boundaries feature collection
india_bbox = ee.FeatureCollection("FAO/GAUL/2015/level2") \
                .filter(ee.Filter.eq('ADM0_NAME', 'United States of America'))

In [ ]:
# Map the function over the FeatureCollection to get centroids
district_centroids = india_bbox.map(add_centroid)

# Get the data as a list of dictionaries for easier conversion to DataFrame
centroid_data = district_centroids.getInfo()['features']


centroid_list = [{
    'County': f['properties']['county_name'].lower(),
    'Centroid_Lon': f['properties']['centroid_lon'],
    'Centroid_Lat': f['properties']['centroid_lat']
} for f in centroid_data]

In [17]:
task = ee.batch.Export.table.toDrive(
    collection=district_centroids,
    description='district_centroids',
    fileNamePrefix='district_centroids',
    fileFormat='CSV',
    selectors=['county_name', 'centroid_lon', 'centroid_lat']  # Specify the columns
)
task.start()


In [18]:
import time

while task.active():
    print("Exporting... Please wait.")
    time.sleep(30)  # Wait 30 seconds before checking again

print("Export completed. Check your Google Drive!")


Exporting... Please wait.
Export completed. Check your Google Drive!


In [20]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
df = pd.read_csv('/content/drive/My Drive/district_centroids.csv')
print(df.head())  # Preview the first few rows

  county_name  centroid_lon  centroid_lat
0     Addison    -73.140962     44.030747
1   Caledonia    -72.102425     44.464422
2  Chittenden    -73.080903     44.460838
3       Essex    -71.736954     44.721187
4    Franklin    -72.913815     44.850126


In [8]:
# Convert to a Pandas DataFrame
df = pd.DataFrame(centroid_list)
# Save to CSV
df.to_csv("district_centroids.csv", index=False)

print("District centroids saved to district_centroids.csv")

District centroids saved to district_centroids.csv


In [22]:
# Function to create bounding box coordinates
def create_bounding_box(lat, lon, width=1.0, height=1.0):
    half_width = width / 2
    half_height = height / 2

    # Define the bounding box corners
    bbox = {
        'min_lon': lon - half_width,
        'min_lat': lat - half_height,
        'max_lon': lon + half_width,
        'max_lat': lat + half_height
    }
    return bbox

In [24]:
# Load the CSV file with centroids
#df = pd.read_csv("district_centroids.csv")

# List to store bounding box data
bounding_boxes = []


In [28]:
# Loop through each centroid and calculate bounding box
for index, row in df.iterrows():
    district = row['county_name']
    lat = row['centroid_lat']
    lon = row['centroid_lon']

    bbox = create_bounding_box(lat, lon)
    bounding_boxes.append({
        'County': district,
        'Min_Lon': bbox['min_lon'],
        'Min_Lat': bbox['min_lat'],
        'Max_Lon': bbox['max_lon'],
        'Max_Lat': bbox['max_lat']
    })

# Convert the list of bounding boxes to a DataFrame
bbox_df = pd.DataFrame(bounding_boxes)


In [32]:
# Save to a new CSV
bbox_df.to_csv("county_bounding_boxes.csv", index=False)

print("Bounding boxes saved to county_bounding_boxes.csv")

Bounding boxes saved to county_bounding_boxes.csv


In [33]:
bbox_df.to_csv("/content/drive/My Drive/county_bounding_boxes.csv", index=False)

print("File saved to Google Drive!")

File saved to Google Drive!
